# Final Model

## Module Import

In [1]:
import re
from textblob import TextBlob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler

In [4]:
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, log_loss
import seaborn as sns
import matplotlib.pyplot as plt

## Data Import

In [6]:
train_df = pd.read_csv('../dataset/train.csv')
test_df = pd.read_csv('../dataset/test.csv')
submission_df = pd.read_csv('../dataset/sample_submission.csv')

In [5]:
significant_features = [
    'len_diff', 'punc_diff',
    'sent_count_diff', 
    'lexical_div_diff',
    'repetition_diff', 
    # 'subjectivity_diff',
    'comma_ratio_diff', 
    # 'avg_word_len_diff'
]

target_words = [
    'company', 'brace', 'knee', 'progression', 
    'apologize', 'sorry'
]

In [7]:
def get_features(df):
    df['len_a'] = df['response_a'].str.len()
    df['len_b'] = df['response_b'].str.len()
    df['punc_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    df['punc_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[!?,;:]', str(x))))
    df['sent_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[.!?]', str(x))))
    df['sent_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[.!?]', str(x))))

    def ling_feat(text):
        words = re.findall(r'\b\w+\b', str(text).lower())
        uniq = set(words)
        lex_div = len(uniq)/(len(words)+1e-9)
        repetition = 1 - len(uniq)/(len(words)+1e-9)
        blob = TextBlob(str(text))
        subj = blob.sentiment.subjectivity
        return lex_div, repetition, subj
    
    for side in ['a', 'b']:
        df[[f'lex_{side}', f'rep_{side}', f'subj_{side}']] = df[f'response_{side}'].apply(
            lambda x: pd.Series(ling_feat(x))
        )
    
    df['comma_a'] = df['response_a'].apply(lambda x: len(re.findall(r'[;,]', str(x))) / (len(x.split()) + 1e-9))
    df['comma_b'] = df['response_b'].apply(lambda x: len(re.findall(r'[;,]', str(x))) / (len(x.split()) + 1e-9))
    # df['avglen_a'] = df['response_a'].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if len(str(x).split()) > 0 else 0)
    # df['avglen_b'] = df['response_b'].apply(lambda x: np.mean([len(w) for w in str(x).split()]) if len(str(x).split()) > 0 else 0)

    # diff features
    df['len_diff'] = df['len_a'] - df['len_b']
    df['punc_diff'] = df['punc_a'] - df['punc_b']
    df['sent_count_diff'] = df['sent_a'] - df['sent_b']
    df['lexical_div_diff'] = df['lex_a'] - df['lex_b']
    df['repetition_diff'] = df['rep_a'] - df['rep_b']
    # df['subjectivity_diff'] = df['subj_a'] - df['subj_b']
    df['comma_ratio_diff'] = df['comma_a'] - df['comma_b']
    # df['avg_word_len_diff'] = df['avglen_a'] - df['avglen_b']
    
    # --- keyword presence ---
    text_cols = df[['prompt', 'response_a', 'response_b']].astype(str).agg(' '.join, axis=1)
    for word in target_words:
        df[f'contains_{word}'] = text_cols.str.contains(fr'\b{word}\b', case=False, na=False).astype(int)

    return df

In [8]:
train_df = get_features(train_df)
test_df = get_features(test_df)

In [9]:
train_df['label'] = np.select(
    [train_df['winner_model_a']==1, train_df['winner_model_b']==1, train_df['winner_tie']==1],
    [0,1,2]
)

In [10]:
keyword_features = [f'contains_{w}' for w in target_words]
all_features = significant_features + keyword_features

In [11]:
X = train_df[all_features]
y = train_df['label']

In [12]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer

# 분석 및 혼합 스케일링(열별로 StandardScaler / RobustScaler / 그대로 유지)

# X, all_features 변수가 이미 존재한다고 가정
df = X.copy()

# 기본 통계량 & 이상치/분산 계산
summary = []
for col in df.columns:
  vals = df[col].dropna().astype(float)
  n_unique = df[col].nunique(dropna=True)
  is_binary = set(vals.unique()).issubset({0.0, 1.0}) or n_unique == 2
  var = vals.var()
  skew = vals.skew()
  q1, q3 = vals.quantile(0.25), vals.quantile(0.75)
  iqr = q3 - q1
  lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
  outlier_mask = (vals < lower) | (vals > upper)
  outlier_ratio = outlier_mask.sum() / max(len(vals), 1)
  low_variance = var < 1e-4  # 임계값은 데이터에 따라 조정 가능
  high_skew = abs(skew) > 1.0

  # 스케일러 권장: 이진 변수는 그대로, 이상치가 많거나 왜도가 큰 경우 Robust, 아니면 Standard
  if is_binary:
    recommended = 'passthrough (binary)'
  elif outlier_ratio > 0.05 or high_skew:
    recommended = 'RobustScaler'
  else:
    recommended = 'StandardScaler'

  summary.append({
    'feature': col,
    'dtype': str(df[col].dtype),
    'n_unique': n_unique,
    'is_binary': is_binary,
    'var': var,
    'skew': skew,
    'q1': q1,
    'q3': q3,
    'iqr': iqr,
    'outlier_ratio': outlier_ratio,
    'low_variance': low_variance,
    'recommended': recommended
  })

summary_df = pd.DataFrame(summary).set_index('feature')
print("Feature summary (first rows):")
print(summary_df[['dtype', 'is_binary', 'n_unique', 'var', 'skew', 'outlier_ratio', 'low_variance', 'recommended']].head(20))

# 그룹 분류
binary_cols = summary_df[summary_df['is_binary']].index.tolist()
robust_cols = summary_df[summary_df['recommended'] == 'RobustScaler'].index.difference(binary_cols).tolist()
std_cols = summary_df[summary_df['recommended'] == 'StandardScaler'].index.difference(binary_cols).tolist()

print("\nGroups:")
print("  binary_cols:", binary_cols)
print("  robust_cols:", robust_cols)
print("  std_cols:", std_cols)

# ColumnTransformer 구성 (빈 그룹은 생략)
transformers = []
if std_cols:
  transformers.append(('std', StandardScaler(), std_cols))
if robust_cols:
  transformers.append(('robust', RobustScaler(), robust_cols))
# passthrough for binary (유지)
if binary_cols:
  transformers.append(('passthrough_binary', 'passthrough', binary_cols))

if not transformers:
  raise RuntimeError("No features to transform. Check X/all_features.")

col_transformer = ColumnTransformer(transformers, remainder='drop', sparse_threshold=0)

# fit + transform -> DataFrame으로 복원
X_scaled_arr = col_transformer.fit_transform(df)
# ColumnTransformer 순서를 이용해 컬럼명 재구성
out_cols = []
for name, _, cols in transformers:
  if cols == 'passthrough' or name.startswith('passthrough'):
    # passthrough 사용한 경우, 실제 컬럼 순서가 입력 df에서 유지되므로 cols는 리스트로 받음
    out_cols.extend(cols if isinstance(cols, (list, tuple)) else list(cols))
  else:
    out_cols.extend(cols)
X_scaled_df = pd.DataFrame(X_scaled_arr, columns=out_cols, index=df.index)

Feature summary (first rows):
                        dtype  is_binary  n_unique           var        skew  \
feature                                                                        
len_diff                int64      False      6041  1.205707e+06   -1.124580   
punc_diff               int64      False       313  3.658851e+02    6.378323   
sent_count_diff         int64      False       255  1.972592e+02   -2.359540   
lexical_div_diff      float64      False     55950  3.080705e-02    0.025245   
repetition_diff       float64      False     55950  3.080705e-02   -0.025245   
comma_ratio_diff      float64      False     51816  1.734652e+00 -165.425305   
contains_company        int64       True         2  4.487974e-02    4.275875   
contains_brace          int64       True         2  2.879821e-03   18.527488   
contains_knee           int64       True         2  3.778502e-03   16.145383   
contains_progression    int64       True         2  7.699548e-03   11.219919   
contains_a

In [13]:
X_scaled_df

,comma_ratio_diff,len_diff,lexical_div_diff,punc_diff,repetition_diff,sent_count_diff,contains_company,contains_brace,contains_knee,contains_progression,contains_apologize,contains_sorry
0,-0.509879,3.952550,-0.550103,3.333333,0.550103,3.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.247947,-0.634638,-0.713681,-0.750000,0.713681,-1.777778,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.727202,-1.084223,0.816672,-1.250000,-0.816672,-0.666667,0.0,0.0,0.0,0.0,0.0,0.0
3,0.382777,1.921708,-0.593601,1.666667,0.593601,1.222222,0.0,0.0,0.0,0.0,0.0,0.0
4,0.069289,0.626335,-0.709755,0.833333,0.709755,0.222222,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
57472,1.690206,-0.190985,1.050461,0.250000,-1.050461,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
57473,0.436278,-0.007117,0.239260,0.000000,-0.239260,-1.222222,0.0,0.0,0.0,0.0,0.0,0.0
57474,-0.402496,8.451957,-0.398586,4.416667,0.398586,10.111111,0.0,0.0,0.0,0.0,0.0,0.0
57475,0.232915,-0.633452,0.009375,-0.333333,-0.009375,-0.666667,0.0,0.0,0.0,0.0,0.0,0.0


## LoRA Embedding Extraction